# Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define dataset path
train_dataset_path = '/content/drive/My Drive/Dataset_Applied_Project/Splitted_Data/train'

test_dataset_path = '/content/drive/My Drive/Dataset_Applied_Project/Splitted_Data/test'

# List the contents of the dataset directory
label_types_train = os.listdir(train_dataset_path)
print(label_types_train)

label_types_test = os.listdir(test_dataset_path)
print(label_types_test)

['crash', 'normal']
['crash', 'normal']


# Preparing Train Data

In [4]:
rooms = []

# Iterate over items (labels) in dataset_path
for item in os.listdir(train_dataset_path):
    # Get all the file names in the current label directory
    all_rooms = os.listdir(os.path.join(train_dataset_path, item))

    # Add file names to the rooms list along with their respective labels
    for room in all_rooms:
        rooms.append((item, os.path.join(train_dataset_path, item, room)))

# Build a DataFrame from the list
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())


     tag                                         video_name
0  crash  /content/drive/My Drive/Dataset_Applied_Projec...
1  crash  /content/drive/My Drive/Dataset_Applied_Projec...
2  crash  /content/drive/My Drive/Dataset_Applied_Projec...
3  crash  /content/drive/My Drive/Dataset_Applied_Projec...
4  crash  /content/drive/My Drive/Dataset_Applied_Projec...
         tag                                         video_name
2395  normal  /content/drive/My Drive/Dataset_Applied_Projec...
2396  normal  /content/drive/My Drive/Dataset_Applied_Projec...
2397  normal  /content/drive/My Drive/Dataset_Applied_Projec...
2398  normal  /content/drive/My Drive/Dataset_Applied_Projec...
2399  normal  /content/drive/My Drive/Dataset_Applied_Projec...


In [5]:
df = train_df.loc[:,['video_name','tag']]
df.to_csv('train.csv')

# Preparing Test Data

In [6]:
rooms = []

# Iterate over items (labels) in the test dataset path
for item in label_types_test:
    # Get all the file names in the current label directory
    all_rooms = os.listdir(os.path.join(test_dataset_path, item))

    # Add file names to the rooms list along with their respective labels
    for room in all_rooms:
        rooms.append((item, os.path.join(test_dataset_path, item, room)))

# Build a DataFrame from the list
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

# Save the DataFrame to a CSV file
test_df.to_csv('test.csv', index=False)


     tag                                         video_name
0  crash  /content/drive/My Drive/Dataset_Applied_Projec...
1  crash  /content/drive/My Drive/Dataset_Applied_Projec...
2  crash  /content/drive/My Drive/Dataset_Applied_Projec...
3  crash  /content/drive/My Drive/Dataset_Applied_Projec...
4  crash  /content/drive/My Drive/Dataset_Applied_Projec...
        tag                                         video_name
595  normal  /content/drive/My Drive/Dataset_Applied_Projec...
596  normal  /content/drive/My Drive/Dataset_Applied_Projec...
597  normal  /content/drive/My Drive/Dataset_Applied_Projec...
598  normal  /content/drive/My Drive/Dataset_Applied_Projec...
599  normal  /content/drive/My Drive/Dataset_Applied_Projec...


In [7]:
!pip install tensorflow-docs


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.5/182.5 kB 4.3 MB/s eta 0:00:00


In [8]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import imageio
import cv2

# Data preparation

In [9]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 2400
Total videos for testing: 600


,Unnamed: 0,video_name,tag
1427,1427,/content/drive/My Drive/Dataset_Applied_Projec...,normal
246,246,/content/drive/My Drive/Dataset_Applied_Projec...,crash
1901,1901,/content/drive/My Drive/Dataset_Applied_Projec...,normal
1182,1182,/content/drive/My Drive/Dataset_Applied_Projec...,crash
426,426,/content/drive/My Drive/Dataset_Applied_Projec...,crash
506,506,/content/drive/My Drive/Dataset_Applied_Projec...,crash
2185,2185,/content/drive/My Drive/Dataset_Applied_Projec...,normal
2076,2076,/content/drive/My Drive/Dataset_Applied_Projec...,normal
2153,2153,/content/drive/My Drive/Dataset_Applied_Projec...,normal
1644,1644,/content/drive/My Drive/Dataset_Applied_Projec...,normal


# Feed the videos to a network:


In [10]:

IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

   ### Feature Extraction

In [11]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87910968/87910968 [==============================] - 1s 0us/step


### Label Encoding
StringLookup layer encode the class labels as integers.

In [12]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['crash', 'normal']


array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

Finally, we can put all the pieces together to create our data processing utility.

In [13]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

MAX_SEQ_LENGTH = 50
NUM_FEATURES = 2048

In [14]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()

    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values

    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

# MAX_SEQ_LENGTH = 50, NUM_FEATURES = 2048.

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 61ms/step
Frame features in train set: (2400, 50, 2048)
Frame masks in train set: (2400, 50)
train_labels in train set: (2400, 1)
test_labels in train set: (600, 1)


# The sequence model
Now, we can feed this data to a sequence model consisting of recurrent layers like GRU.

In [15]:
# Utility for sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
53/53 [==============================] - ETA: 0s - loss: 0.4531 - accuracy: 0.8065
Epoch 1: val_loss improved from inf to 0.55926, saving model to ./tmp/video_classifier
53/53 [==============================] - 15s 84ms/step - loss: 0.4531 - accuracy: 0.8065 - val_loss: 0.5593 - val_accuracy: 0.7944
Epoch 2/30
52/53 [============================>.] - ETA: 0s - loss: 0.2690 - accuracy: 0.9075
Epoch 2: val_loss improved from 0.55926 to 0.27076, saving model to ./tmp/video_classifier
53/53 [==============================] - 1s 25ms/step - loss: 0.2691 - accuracy: 0.9071 - val_loss: 0.2708 - val_accuracy: 0.9319
Epoch 3/30
52/53 [============================>.] - ETA: 0s - loss: 0.1856 - accuracy: 0.9387
Epoch 3: val_loss did not improve from 0.27076
53/53 [==============================] - 1s 24ms/step - loss: 0.1843 - accuracy: 0.9393 - val_loss: 0.3944 - val_accuracy: 0.8764
Epoch 4/30
52/53 [============================>.] - ETA: 0s - loss: 0.1290 - accuracy: 0.9621
Epoch 4:

# Testing our model

In [16]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)


Test video path: /content/drive/My Drive/Dataset_Applied_Project/Splitted_Data/test/crash/crash_811.mp4
1/1 [==============================] - 2s 2s/step
  crash: 96.96%
  normal:  3.04%


In [17]:
from IPython.display import HTML

HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src="/content/drive/MyDrive/car_dataset_small/test/crash/crash_45.mp4" type="video/mp4" style="height:300px;width:300px">
    </video>
""")



# Saving our model

In [18]:
# Save the model's weights to a file on Google Drive
sequence_model.save_weights("model_weights.h5")

# Save the model's architecture as JSON
model_json = sequence_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

